In [0]:

import pandas as pd

from textblob import TextBlob
from nltk import sentiment
from sklearn.metrics import confusion_matrix,accuracy_score

In [0]:
data=pd.read_json('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz',lines=True)
df = pd.read_csv("https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv",error_bad_lines=False)



data
df


,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...
...,...,...,...,...,...
5108,twitter,irrelevant,126855687060987904,Thu Oct 20 03:02:07 +0000 2011,me re copè con #twitter
5109,twitter,irrelevant,126855171702661120,Thu Oct 20 03:00:04 +0000 2011,Buenas noches genteeee :) #twitter los quieroo...
5110,twitter,irrelevant,126854999442587648,Thu Oct 20 02:59:23 +0000 2011,#twitter tiene la mala costumbre de ponerce bn...
5111,twitter,irrelevant,126854818101858304,Thu Oct 20 02:58:40 +0000 2011,Oi @flaviasansi. Muito bem vinda ao meu #Twitt...


In [0]:
df=df[df['Sentiment']!='irrelevant']
df['Sentiment'].value_counts()

neutral     2333
negative     572
positive     519
Name: Sentiment, dtype: int64

In [0]:
import nltk,re

In [0]:
# from nltk import sentiment

In [0]:
sent=data['reviewText'][0]

In [0]:
def txt_blob(text):
  txt_blob=TextBlob(text).sentiment
  polarity=txt_blob.polarity
  if polarity==0:
    return "neutral"
  elif polarity>0:
    return "positive"
  else:
    return "negative"


In [0]:
data['txt_blob_senti']=data.reviewText.apply(txt_blob)
data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,sent_dict,max_sent,txt_blob_senti
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004","{'neg': 0.039, 'neu': 0.781, 'pos': 0.18, 'com...",compound,positive
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007","{'neg': 0.029, 'neu': 0.872, 'pos': 0.099, 'co...",compound,positive
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011","{'neg': 0.154, 'neu': 0.72, 'pos': 0.126, 'com...",neu,positive
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006","{'neg': 0.036, 'neu': 0.814, 'pos': 0.151, 'co...",compound,positive
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013","{'neg': 0.022, 'neu': 0.823, 'pos': 0.154, 'co...",compound,positive


In [0]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [0]:

# sent =data['reviewText'][0]
# blob(sent)
sid=sentiment.vader.SentimentIntensityAnalyzer()

In [0]:
def sent_classifier(text):
  sid
  pol_score = sid.polarity_scores(text)
  if pol_score['compound'] >= 0.05:
    return "positive"
  elif pol_score['compound'] <= -0.05:
    return "negative"
  else:
    return "neutral"

In [0]:
data['sent_SIA']=data.reviewText.apply(sent_classifier)

In [0]:
data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,sent_dict,max_sent,txt_blob_senti,sent_SIA
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004","{'neg': 0.039, 'neu': 0.781, 'pos': 0.18, 'com...",compound,positive,positive
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007","{'neg': 0.029, 'neu': 0.872, 'pos': 0.099, 'co...",compound,positive,positive
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011","{'neg': 0.154, 'neu': 0.72, 'pos': 0.126, 'com...",neu,positive,negative
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006","{'neg': 0.036, 'neu': 0.814, 'pos': 0.151, 'co...",compound,positive,positive
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013","{'neg': 0.022, 'neu': 0.823, 'pos': 0.154, 'co...",compound,positive,positive


In [0]:
df['TweetText'].head()

0    Now all @Apple has to do is get swype on the i...
1    @Apple will be adding more carrier support to ...
2    Hilarious @youtube video - guy does a duet wit...
3    @RIM you made it too easy for me to switch to ...
4    I just realized that the reason I got into twi...
Name: TweetText, dtype: object

In [0]:
sent1=df['TweetText'][1]
a=sid.polarity_scores(sent1)
a

{'compound': 0.4549, 'neg': 0.0, 'neu': 0.801, 'pos': 0.199}

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
df['tweet_sentiments']=df.TweetText.apply(sid.polarity_scores)

In [0]:
df['tweet_SIA']=df.TweetText.apply(sent_classifier)

In [0]:
df['tweettxt_txtblob']=df.TweetText.apply(txt_blob)
a=confusion_matrix(df['Sentiment'],df['tweettxt_txtblob'])
a

array([[ 234,  171,  167],
       [ 201, 1213,  919],
       [  38,  124,  357]])

In [0]:
b=confusion_matrix(df['Sentiment'],df['tweet_SIA'])
b

array([[ 291,  138,  143],
       [ 302, 1234,  797],
       [  47,   99,  373]])

In [0]:
import operator

In [0]:
def sent_finder(dictionary):
            return max(dictionary.items(), key=operator.itemgetter(1))[0]

In [0]:
df['max_sent']=df.tweet_sentiments.apply(sent_finder)

In [0]:
df.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,tweet_sentiments,max_sent,tweettxt_txtblob,tweet_SIA
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neu,neutral,neutral
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,"{'neg': 0.0, 'neu': 0.801, 'pos': 0.199, 'comp...",neu,positive,positive
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,"{'neg': 0.0, 'neu': 0.591, 'pos': 0.409, 'comp...",compound,positive,positive
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,"{'neg': 0.0, 'neu': 0.814, 'pos': 0.186, 'comp...",neu,positive,positive
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,"{'neg': 0.0, 'neu': 0.791, 'pos': 0.209, 'comp...",neu,positive,positive


In [0]:
data['sent_dict']=data.reviewText.apply(sid.polarity_scores)

In [0]:
data['max_sent']=data.sent_dict.apply(sent_finder)

In [0]:
data.head()


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,sent_dict,max_sent,txt_blob_senti
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004","{'neg': 0.039, 'neu': 0.781, 'pos': 0.18, 'com...",compound,pos
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007","{'neg': 0.029, 'neu': 0.872, 'pos': 0.099, 'co...",compound,pos
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011","{'neg': 0.154, 'neu': 0.72, 'pos': 0.126, 'com...",neu,pos
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006","{'neg': 0.036, 'neu': 0.814, 'pos': 0.151, 'co...",compound,pos
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013","{'neg': 0.022, 'neu': 0.823, 'pos': 0.154, 'co...",compound,pos


In [0]:
# data['Textblob_polarity']=data.reviewText.apply(blob)
!pip install glove_python

     |████████████████████████████████| 266kB 2.8MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700330 sha256=c784072e5a52faa20b31d6945b03c57b04a8f2199d165e7728c5bf4c0d27689d
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [0]:
from glove import Glove
from glove import Corpus


In [0]:
sent=data['reviewText'][0]

In [0]:
sent=list(sent)

In [0]:
glove=Glove()
corp=Corpus()

In [0]:
# glove_para=glove.most_similar(sent)
corp.fit(sent)

TypeError: ignored